# Filtro de imagenes Landsat google Earth Engine

In [1]:
import ee
ee.Initialize()

## Coleccion Landsat

In [2]:
# Coleccion de Landsat Reflectacia de superficie
L5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") # Landsat 5 TM RS LANDSAT/LT05/C01/T1_SR
L7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") # Landsat 7 ETM RS LANDSAT/LE07/C01/T1_SR
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") # Landsat 8 OLI TIRS RS LANDSAT/LC08/C01/T1_SR
L8_L2 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") # Landsat 8 OLI TIRS RS LEVEL 2
L7_L2 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") # Landsat 7 ETM+ RS LEVEL 2

## Filtro Path y Row

In [3]:
# Método 1
L5_F1 = L5.filter(ee.Filter.eq('WRS_PATH', 3))\
          .filter(ee.Filter.eq('WRS_ROW', 69))

In [4]:
# Método 2    
L5_F2 = L5.filterMetadata('WRS_PATH', 'equals', 3)\
          .filterMetadata('WRS_ROW', 'equals', 69)


In [6]:
# Conteo de imagenes
count = L5_F2.size()
print("Cantidad de imagenes:",count.getInfo())

Cantidad de imagenes: 252


## Filtro zona estudio SHP

In [7]:
import os
import glob

In [8]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP'

In [9]:
lista = glob.glob("*.shp")
lista

['Cuenca_Geo.shp', 'ZONA_geo.shp', 'Zona_Geografica.shp', 'Zona_UTM_19S.shp']

In [10]:
import geopandas as gpd

In [11]:
# cargar un shp
zona = gpd.read_file("ZONA_geo.shp")
zona

,id,geometry
0,1,"POLYGON ((-70.14663 -12.74826, -69.84347 -12.7..."


In [12]:
# Crear un featureCollecion mediante GeoJSON
ee_zona = ee.FeatureCollection(zona.__geo_interface__)

In [13]:
type(ee_zona)

ee.featurecollection.FeatureCollection

### Aplicamos el filtro a la Zona

In [14]:
L5_Zona = L5.filterBounds(ee_zona)

In [15]:
# Conteo de imagenes
count = L5_Zona.size()
print("Cantidad de imagenes:",count.getInfo())

Cantidad de imagenes: 369


## Filtro de nubosidad

In [16]:
# Puede aplicar al filtro Path y Row o segun zona
L5_Nubosidad = L5_F1.filterMetadata('CLOUD_COVER', 'less_than', 20)

In [17]:
# Conteo de imagenes
count = L5_Nubosidad.size()
print("Cantidad de imagenes:",count.getInfo())

Cantidad de imagenes: 68


## Filtro fecha

In [18]:
L5_Fecha = L5_Nubosidad.filterDate('2008-01-01','2009-12-31')

In [19]:
# Conteo de imagenes
count = L5_Fecha.size()
print("Cantidad de imagenes:",count.getInfo())

Cantidad de imagenes: 7


In [29]:
# Imprimir la lista de ID Imagenes
catalogo_L5 = L5_Fecha.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
catalogo_L5

['LT05_003069_20080505',
 'LT05_003069_20080606',
 'LT05_003069_20080825',
 'LT05_003069_20080910',
 'LT05_003069_20090524',
 'LT05_003069_20090812',
 'LT05_003069_20090828']

In [30]:
type(catalogo_L5)

list

## Filtro en una sola linea

In [32]:
# Filtro por Path y Row
L5_1 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")\
        .filterDate('2008-01-01','2009-12-31')\
        .filterMetadata('CLOUD_COVER', 'less_than',20)\
        .filterMetadata('WRS_PATH', 'equals', 3)\
        .filterMetadata('WRS_ROW', 'equals', 69)

In [31]:
# Filtro por zona de estudio
L5_2 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")\
        .filterDate('2008-01-01','2009-12-31')\
        .filterMetadata('CLOUD_COVER', 'less_than',20)\
        .filterBounds(ee_zona)

In [33]:
# Conteo de imagenes
count_L5_1 = L5_1.size()
count_L5_2 = L5_2.size()
print("Cantidad de imagenes L5_1:", count_L5_1.getInfo())
print("Cantidad de imagenes L5_2:", count_L5_2.getInfo())

Cantidad de imagenes L5_1: 7
Cantidad de imagenes L5_2: 9


In [34]:
# Imprimir la lista de ID Imagenes
catalogo_L5_1 = L5_1.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
catalogo_L5_2 = L5_2.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print("ID catalogo_L5_1: ",catalogo_L5_1)
print("ID catalogo_L5_2: ",catalogo_L5_2)

ID catalogo_L5_1:  ['LT05_003069_20080505', 'LT05_003069_20080606', 'LT05_003069_20080825', 'LT05_003069_20080910', 'LT05_003069_20090524', 'LT05_003069_20090812', 'LT05_003069_20090828']
ID catalogo_L5_2:  ['LT05_002069_20080903', 'LT05_002069_20090906', 'LT05_003069_20080505', 'LT05_003069_20080606', 'LT05_003069_20080825', 'LT05_003069_20080910', 'LT05_003069_20090524', 'LT05_003069_20090812', 'LT05_003069_20090828']
